In [15]:
from cobra import Model, Reaction, Metabolite
import tempfile
from pprint import pprint 
from cobra.io import write_sbml_model, validate_sbml_model

### Model, Reactions, and Metabolites

In [16]:
model = Model('example_model') #creates new model object - model is the whole system 

reaction = Reaction('R_3OAS140') #creates new reaction object - reaction is, you guessed it, a single reaction 
reaction.name = '3 oxoacyl acyl carrier protein synthase n C140' 
reaction.subsystem = 'Cell Envelope Biosynthesis'
reaction.lower_bound = 0 #lowest rate at which the reaction can proceed, measured in mmol/gDW/h (millimoles per gram dry weight per hour)
reaction.upper_bound = 1000 
reaction

Reaction identifier,R_3OAS140
Name,3 oxoacyl acyl carrier protein synthase n C140
Memory address,0x7f4c9eac4590
Stoichiometry,--> -->
GPR,
Lower bound,0
Upper bound,1000


In [17]:
model

Name,example_model
Memory address,7f4ca2cc3da0
Number of metabolites,0
Number of reactions,0
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [18]:
#creates metabolites - # reaction id's have to be Systems Biology Markup Language (SBML) compliant. 
ACP_c = Metabolite(
    'ACP_c',
    formula='C11H21N2O7PRS',
    name='acyl-carrier-protein',
    compartment='c')
omrsACP_c = Metabolite(
    'M3omrsACP_c',
    formula='C25H45N2O9PRS',
    name='3-Oxotetradecanoyl-acyl-carrier-protein',
    compartment='c')
co2_c = Metabolite('co2_c', formula='CO2', name='CO2', compartment='c')
malACP_c = Metabolite(
    'malACP_c',
    formula='C14H22N2O10PRS',
    name='Malonyl-acyl-carrier-protein',
    compartment='c')
h_c = Metabolite('h_c', formula='H', name='H', compartment='c')
ddcaACP_c = Metabolite(
    'ddcaACP_c',
    formula='C23H43N2O8PRS',
    name='Dodecanoyl-ACP-n-C120ACP',
    compartment='c')

In [19]:
reaction.add_metabolites({ #this is a dictionary of the metabolites and their stoichiometric coeffiecients. Notice how the metabolites are defined previously. 
    malACP_c: -1.0,
    h_c: -1.0,
    ddcaACP_c: -1.0,
    co2_c: 1.0,
    ACP_c: 1.0,
    omrsACP_c: 1.0
})

reaction

Reaction identifier,R_3OAS140
Name,3 oxoacyl acyl carrier protein synthase n C140
Memory address,0x7f4c9eac4590
Stoichiometry,ddcaACP_c + h_c + malACP_c --> ACP_c + M3omrsACP_c + co2_c Dodecanoyl-ACP-n-C120ACP + H + Malonyl-acyl-carrier-protein --> acyl-carrier-protein + 3-Oxotetradecanoyl-acyl-carrier-protein + CO2
GPR,
Lower bound,0
Upper bound,1000


In [20]:
reaction.gene_reaction_rule = '( STM2378 or STM1197 )' #We saw this previously - this just adds genes involved in the reaction. 
reaction.genes

frozenset({<Gene STM1197 at 0x7f4c9e76ffb0>, <Gene STM2378 at 0x7f4c9e76e600>})

In [21]:
print(len(model.reactions), 'reactions') 
print(len(model.metabolites), 'metabolites') 
print(len(model.genes), 'genes') 

#This is just to show nothing we have not added all of the reaction definitions to the model. 


0 reactions
0 metabolites
0 genes


In [22]:
model.add_reactions([reaction]) #this line specifically adds the defined reaction to the model 

print(len(model.reactions), 'reactions') 
print(len(model.metabolites), 'metabolites') 
print(len(model.genes), 'genes') 

#This is to show that now everything we defined as been added to the model. 

1 reactions
6 metabolites
2 genes


In [23]:
print('Model Reactions')
print('--------------')
for x in model.reactions:
    print('%s : %s' % (x.id, x.reaction))

Model Reactions
--------------
R_3OAS140 : ddcaACP_c + h_c + malACP_c --> ACP_c + M3omrsACP_c + co2_c


In [24]:
print('Metabolites')
print('-----------')
for x in model.metabolites:
    print('%9s : %s' % (x.id, x.formula))

Metabolites
-----------
 malACP_c : C14H22N2O10PRS
      h_c : H
ddcaACP_c : C23H43N2O8PRS
    co2_c : CO2
    ACP_c : C11H21N2O7PRS
M3omrsACP_c : C25H45N2O9PRS


In [25]:
print('Genes')
print('-----------')
for x in model.genes:
    associated_ids = (i.id for i in x.reactions)
    print('%s is associated with reactions: %s' % (x.id, ', '.join(associated_ids)))

Genes
-----------
STM1197 is associated with reactions: R_3OAS140
STM2378 is associated with reactions: R_3OAS140


### Objective and Model Validation

In [26]:
model.objective = 'R_3OAS140' #sets the objective function of the model to be the reaction we just added.
print(model.objective.expression) #Algebraic expression of the model's objective function
print(model.objective.direction) #indicates the objective function is set to be maximized. 

1.0*R_3OAS140 - 1.0*R_3OAS140_reverse_60acb
max


In [28]:
with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml: #creates and returns a temporary file
    write_sbml_model(model, filename=f_sbml.name) #writes the cobra model to the file name 
    report = validate_sbml_model(filename=f_sbml.name) #Validates the model and returns it along with a list of errors. 

pprint(report) #There are no errors in the model. 

(<Model example_model at 0x7f4c9eab2f90>,
 {'COBRA_CHECK': [],
  'COBRA_ERROR': [],
  'COBRA_FATAL': [],
  'COBRA_WARNING': [],
  'SBML_ERROR': [],
  'SBML_FATAL': [],
  'SBML_SCHEMA_ERROR': [],
  'SBML_WARNING': []})


### Exchanges, Sinks, and Demands

In [29]:
# boudnary reactions represent the transport of metabolites between cells and their environment. There are three types; exchange, demand, and sink
print('exchanges', model.exchanges)
print('demands', model.demands)
print('sinks', model.sinks)

There are no boundary reactions in this model. Therefore, specific types of boundary reactions such as 'exchanges', 'demands' or 'sinks' cannot be identified.
There are no boundary reactions in this model. Therefore, specific types of boundary reactions such as 'exchanges', 'demands' or 'sinks' cannot be identified.
There are no boundary reactions in this model. Therefore, specific types of boundary reactions such as 'exchanges', 'demands' or 'sinks' cannot be identified.


exchanges []
demands []
sinks []


In [32]:
model.add_metabolites([Metabolite('glycogen_c', name='glycogen', compartment='c'), Metabolite('co2_e', name='CO2', compartment='e')]) #  c = cytosol, e = extracellular 

model.metabolites

[<Metabolite malACP_c at 0x7f4c9eab0fb0>,
 <Metabolite h_c at 0x7f4c9eab0da0>,
 <Metabolite ddcaACP_c at 0x7f4c9e76f830>,
 <Metabolite co2_c at 0x7f4c9eab31d0>,
 <Metabolite ACP_c at 0x7f4c9eab25d0>,
 <Metabolite M3omrsACP_c at 0x7f4c9eab2300>,
 <Metabolite glycogen_c at 0x7f4c9e7b7ad0>,
 <Metabolite co2_e at 0x7f4c9e8ab620>]

In [33]:
model.add_boundary(model.metabolites.get_by_id('co2_e'), type='exchange') #adds a boundary reaction for a specific type of metabolite 

Reaction identifier,EX_co2_e
Name,CO2 exchange
Memory address,0x7f4ca52e7b30
Stoichiometry,co2_e <=> CO2 <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [34]:
model.add_boundary(model.metabolites.get_by_id('glycogen_c'), type = 'sink')

Reaction identifier,SK_glycogen_c
Name,glycogen sink
Memory address,0x7f4c9e8c48c0
Stoichiometry,glycogen_c <=> glycogen <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [35]:
print('exchanges', model.exchanges) #These just show we've added an exchange and a sink reaction 
print('sinks', model.sinks)
print('demands', model.demands)

exchanges [<Reaction EX_co2_e at 0x7f4ca52e7b30>]
sinks [<Reaction SK_glycogen_c at 0x7f4c9e8c48c0>]
demands []


In [36]:
model.boundary #easier way to get all boundary reactions. 

[<Reaction EX_co2_e at 0x7f4ca52e7b30>,
 <Reaction SK_glycogen_c at 0x7f4c9e8c48c0>]